In [1]:
import pandas as pd
import time
import sys
def player_data(start_year,end_year,ps=False):
    if ps == False:
        season_type='leagues'
    else:
        season_type='playoffs'
    frames=[]
    for year in range(start_year,end_year+1):
        url = 'https://www.basketball-reference.com/'+season_type+'/NBA_'+str(year)+'_per_poss.html'
        df=pd.read_html(url)[0]
        df=df[df.Player!='Player']
        time.sleep(1)
        url2='https://www.basketball-reference.com/'+season_type+'/NBA_'+str(year)+'_advanced.html'
        df2=pd.read_html(url2)[0]
        df2=df2[df2.Player!='Player']

        

        year_df=df.merge(df2)
        unnamed=[col for col in year_df if 'unnamed' in col.lower()]
        year_df.drop(columns=unnamed,inplace=True)
        year_df['year']=year
        frames.append(year_df)
        print(year)

        time.sleep(2)
    return pd.concat(frames)
start_year=1974
end_year=2024

player_stats=player_data(start_year,end_year)

player_stats.to_csv('player_stats.csv',index=False)
player_stats.to_csv('player_stats_raw.csv',index=False)

player_stats_ps=player_data(start_year,end_year,ps=True)
player_stats_ps.to_csv('player_stats_ps.csv',index=False)
player_stats_ps.to_csv('player_stats_ps_raw.csv',index=False)


1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990


HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
def league_average(league='leagues'):
    time.sleep(1)
    league_url='https://www.basketball-reference.com/'+league+'/NBA_stats_per_poss.html'
    frames=pd.read_html(league_url)
    league = frames[0]
    league.columns=league.columns.droplevel()
    league=league[league.Season!='Season']

    league2 = frames[1]
    league2.columns=league2.columns.droplevel()
    league2=league2[league2.Season!='Season']
    
    
    return league,league2
league,league2 = league_average()
league.to_csv('league_average.csv',index=False)

league2.to_csv('league_average_ps.csv',index=False)



In [31]:
import unicodedata
import re
def transliterate(text):
    # Normalize to NFKD form (separate characters from diacritics)
    text = unicodedata.normalize('NFKD', text)
    # Remove diacritics
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    # Replace remaining non-ASCII characters
    text = text.encode('ascii', 'ignore').decode('ascii')
    return text

In [34]:
players = pd.read_csv('player_stats_raw.csv')
print(len(players))
players.rename(columns={'Player':'player'},inplace=True)
players['player']=players['player'].str.strip('*')

players['player']=players['player'].apply(transliterate)
player_index=pd.read_csv('index_master.csv')
player_index['player']=player_index['player'].apply(transliterate)

players=players.merge(player_index,on=['player','year'],how='left')
players.drop_duplicates(inplace=True)
print(len(players))
players.to_csv('player_stats.csv',index=False)
players.drop_duplicates()

25751
25811


,Rk,player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,WS,WS/48,OBPM,DBPM,BPM,VORP,year,url,bref_id,nba_id
0,1,Zaid Abdul-Aziz,C,27,HOU,79,NaN,2459,6.2,13.5,...,6.5,0.126,0.9,0.1,1.0,1.8,1974,https://www.basketball-reference.com/players/a...,abdulza01,76002.0
1,2,Kareem Abdul-Jabbar,C,26,MIL,81,NaN,3548,12.2,22.6,...,18.4,0.250,5.0,2.5,7.5,8.6,1974,https://www.basketball-reference.com/players/a...,abdulka01,76003.0
2,3,Don Adams,SF,26,DET,74,NaN,2298,5.9,14.6,...,3.6,0.076,-2.2,0.7,-1.4,0.3,1974,https://www.basketball-reference.com/players/a...,adamsdo01,76012.0
3,4,Rick Adelman,PG,27,CHI,55,0.0,618,4.7,12.6,...,0.7,0.053,-3.5,1.8,-1.6,0.1,1974,https://www.basketball-reference.com/players/a...,adelmri01,76015.0
4,5,Lucius Allen,PG,26,MIL,72,NaN,2388,10.0,20.3,...,8.2,0.164,1.6,1.0,2.5,2.7,1974,https://www.basketball-reference.com/players/a...,allenlu01,76027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40114,568,Thaddeus Young,PF,35,PHO,10,0.0,89,6.0,11.5,...,0.3,0.137,-1.5,0.6,-0.9,0.0,2024,https://www.basketball-reference.com/players/y...,youngth01,201152.0
40117,569,Trae Young,PG,25,ATL,54,54.0,1942,10.7,24.9,...,4.6,0.114,4.9,-2.3,2.6,2.2,2024,https://www.basketball-reference.com/players/y...,youngtr01,1629027.0
40118,570,Omer Yurtseven,C,25,UTA,48,12.0,545,8.8,16.3,...,0.7,0.062,-1.6,-1.5,-3.0,-0.1,2024,https://www.basketball-reference.com/players/y...,yurtsom01,1630209.0
40119,571,Cody Zeller,C,31,NOP,43,0.0,320,4.0,9.5,...,0.8,0.124,-2.9,0.3,-2.6,0.0,2024,https://www.basketball-reference.com/players/z...,zelleco01,203469.0


In [36]:
player_index[player_index.player.str.contains('Dennis')]

,player,url,year,bref_id,nba_id
1541,Dennis Hamilton,https://www.basketball-reference.com/players/h...,1968,hamilde01,76926.0
1724,Dennis Hamilton,https://www.basketball-reference.com/players/h...,1969,hamilde01,76926.0
2069,Dennis Awtrey,https://www.basketball-reference.com/players/a...,1971,awtrede01,76078.0
2256,Dennis Stewart,https://www.basketball-reference.com/players/s...,1971,stewade01,78253.0
2312,Dennis Awtrey,https://www.basketball-reference.com/players/a...,1972,awtrede01,76078.0
...,...,...,...,...,...
28010,Dexter Dennis,https://www.basketball-reference.com/players/d...,2024,dennide01,1641926.0
28437,Dennis SchrAder,https://www.basketball-reference.com/players/s...,2024,schrode01,203471.0
28438,Dennis SchrAder,https://www.basketball-reference.com/players/s...,2024,schrode01,203471.0
28439,Dennis SchrAder,https://www.basketball-reference.com/players/s...,2024,schrode01,203471.0


In [20]:
#players = pd.read_csv('player_stats_raw.csv')

players[players.player.str.contains('Karl')]

,Rk,player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,WS,WS/48,OBPM,DBPM,BPM,VORP,year,url,bref_id,nba_id
1365,144,George Karl,PG,25,SAS,29,NaN,251,4.3,12.5,...,-0.2,-0.037,-4.2,-1.6,-5.7,-0.3,1977,https://www.basketball-reference.com/players/k...,karlge01,77223.0
1983,143,George Karl,PG,26,SAS,4,NaN,30,2.9,8.8,...,0.0,-0.048,-4.7,-0.1,-4.7,0.0,1978,https://www.basketball-reference.com/players/k...,karlge01,77223.0
6411,171,Karl Malone,PF,22,UTA,81,76.0,2475,9.5,19.1,...,1.9,0.036,-2.4,0.5,-1.9,0.1,1986,https://www.basketball-reference.com/players/m...,malonka01,252.0
6946,166,Karl Malone,PF,23,UTA,82,82.0,2857,11.9,23.2,...,6.9,0.116,0.3,0.0,0.3,1.7,1987,https://www.basketball-reference.com/players/m...,malonka01,252.0
7533,169,Karl Malone,PF,24,UTA,82,82.0,3198,12.7,24.4,...,10.1,0.152,1.7,-0.2,1.5,2.9,1988,https://www.basketball-reference.com/players/m...,malonka01,252.0
8237,197,Karl Malone,PF,25,UTA,80,80.0,3126,12.7,24.4,...,15.2,0.233,4.3,1.2,5.4,5.9,1989,https://www.basketball-reference.com/players/m...,malonka01,252.0
8968,207,Karl Malone,PF,26,UTA,82,82.0,3122,14.6,26.0,...,15.9,0.245,5.9,0.5,6.4,6.6,1990,https://www.basketball-reference.com/players/m...,malonka01,252.0
9641,212,Karl Malone,PF,27,UTA,82,82.0,3302,12.9,24.5,...,15.5,0.225,4.8,0.6,5.4,6.2,1991,https://www.basketball-reference.com/players/m...,malonka01,252.0
10287,215,Karl Malone,PF,28,UTA,81,81.0,3054,13.1,25.0,...,15.1,0.237,5.2,0.6,5.7,5.9,1992,https://www.basketball-reference.com/players/m...,malonka01,252.0
10914,217,Karl Malone,PF,29,UTA,82,82.0,3099,12.8,23.2,...,15.4,0.238,6.0,1.4,7.3,7.3,1993,https://www.basketball-reference.com/players/m...,malonka01,252.0


In [ ]:
players = pd.read_csv('player_stats_ps_raw.csv')
players.rename(columns={'Player':'player'},inplace=True)
player_index=pd.read_csv('index_master.csv')
players=players.merge(player_index,on=['player','year'])

players.to_csv('player_stats_ps.csv',index=False)
players

In [ ]:
players.dropna(subset='nba_id')